In this notebook I will test applying the gain saved in the data file.

In [1]:
import xarray as xr
import glob
import numpy as np
import sys




In [2]:
sys.path.append("/echopype/echopype")
!pip install ./echopype/.

ERROR: Invalid requirement: './echopype/.'


In [2]:
import echopype as ep
ep.__version__

'0.5.6'

In [3]:
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

## EK60

In [4]:
nc_path = 'F:GLIDER2018/SeaPatches2018_tokt_all/EK60/Seapatches_2018/nc_files/D20180617-T160809.nc'
ed_ek60 = ep.open_converted(nc_path)

In [5]:
ed_ek60

EchoData: standardized raw data from F:GLIDER2018\SeaPatches2018_tokt_all\EK60\Seapatches_2018\nc_files\D20180617-T160809.nc
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.

In [6]:
ds_Sv_ek60 = ep.calibrate.compute_Sv(ed_ek60,waveform_mode='CW')

In [7]:
ds_Sv_ek60

<xarray.Dataset>
Dimensions:                (frequency: 2, ping_time: 3627, range_bin: 1060)
Coordinates:
  * frequency              (frequency) float64 1.8e+04 3.8e+04
  * ping_time              (ping_time) datetime64[ns] 2018-06-17T16:08:09.901...
  * range_bin              (range_bin) int32 0 1 2 3 4 ... 1056 1057 1058 1059
Data variables:
    Sv                     (frequency, ping_time, range_bin) float64 -0.5936 ...
    range                  (frequency, ping_time, range_bin) float64 0.0 ... nan
    sound_speed            (frequency, ping_time) float64 1.474e+03 ... 1.478...
    sound_absorption       (frequency, ping_time) float64 0.003167 ... 0.01035
    sa_correction          (frequency) float64 -0.54 -0.58
    gain_correction        (frequency) float64 23.15 27.13
    equivalent_beam_angle  (frequency) float64 -17.0 -20.8

## EK80

1 channel

In [9]:
nc_path = 'F:GLIDER2018/Echosounder/Echosounder/nc_files/GLIDER_SB_2018_v5-Phase0-D20180625-T173106-0.nc'
ed = ep.open_converted(nc_path)

In [10]:
ed

EchoData: standardized raw data from F:GLIDER2018\Echosounder\Echosounder\nc_files\GLIDER_SB_2018_v5-Phase0-D20180625-T173106-0.nc
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.

In [11]:
ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode='BB', encode_mode='complex')


C:\Users\mbd\anaconda3\envs\phd-env\lib\site-packages\xarray\core\alignment.py:307: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  index = joiner(matching_indexes)
C:\Users\mbd\anaconda3\envs\phd-env\lib\site-packages\xarray\core\alignment.py:307: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  index = joiner(matching_indexes)


In [16]:
np.diff(ds_Sv.range)

array([[0.        , 0.        , 0.        , ..., 0.00592999, 0.00592999,
        0.00592999],
       [0.        , 0.        , 0.        , ..., 0.00592999, 0.00592999,
        0.00592999],
       [0.        , 0.        , 0.        , ..., 0.00592999, 0.00592999,
        0.00592999],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00592999, 0.00592999,
        0.00592999],
       [0.        , 0.        , 0.        , ..., 0.00592999, 0.00592999,
        0.00592999],
       [0.        , 0.        , 0.        , ..., 0.00592999, 0.00592999,
        0.00592999]])

## EK80 with frequency dependant gain cal

Wu-Jung Lee: for BB , the data files sometimes come with an array that encodes freq-dependent cal values (those you get from sphere calibration), and the interpolation of that one is what needs to be implemented, to interpolated to the center frequency given the recorded frequency_start and frequency_end

In [13]:
nc_path = 'F:Test_files/nc_files/D20170912-T234910.nc'
ed = ep.open_converted(nc_path)
ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode='BB', encode_mode='complex')

In [14]:
ed

EchoData: standardized raw data from F:Test_files\nc_files\D20170912-T234910.nc
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.

### Interpolate the frequency to find center frequency ind

In [15]:
freq_nominal = ed.beam.frequency
freq_nominal-ed.vendor.cal_frequency[0] < ed.vendor.cal_frequency[-1]-ed.vendor.cal_frequency[0]

<xarray.DataArray (frequency: 2)>
array([ True, False])
Coordinates:
  * frequency      (frequency) float64 7e+04 2e+05
    cal_frequency  int32 45000

In [16]:
freq_center = (
                ed.beam["frequency_start"]
                + ed.beam["frequency_end"]
            ) / 2
freq_center[1]

<xarray.DataArray (ping_time: 88)>
array([210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000., 210000., 210000., 210000.,
       210000., 210000., 210000., 210000.])
Coordinates:
    frequency  float64 2e+05
  * ping_time  (ping_time) datetime64[ns] 2017-09-12T23:49:10.722999808 ... 2...

In [17]:
freq_nominal = ed.beam.frequency
range(len(freq_nominal))

range(0, 2)

In [18]:
ed.vendor.gain.interp(cal_frequency=freq_center[0])[0][0]

<xarray.DataArray 'gain' ()>
array(27.99637097)
Coordinates:
    cal_channel_id  <U21 'WBT 549762-15 ES70-7C'
    cal_frequency   float64 6.75e+04
    frequency       float64 7e+04
    ping_time       datetime64[ns] 2017-09-12T23:49:10.722999808

## EK80 frequency dependant gain


In [19]:
nc_path = 'F:Test_files/nc_files/D20170912-T234910.nc'
ed = ep.open_converted(nc_path)

In [20]:
ed

EchoData: standardized raw data from F:Test_files\nc_files\D20170912-T234910.nc
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.

In [21]:
ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode='BB', encode_mode='complex')

In [22]:
ds_Sv

<xarray.Dataset>
Dimensions:                (frequency: 2, ping_time: 88, range_bin: 25229)
Coordinates:
  * ping_time              (ping_time) datetime64[ns] 2017-09-12T23:49:10.722...
  * frequency              (frequency) float64 7e+04 2e+05
  * range_bin              (range_bin) int64 0 1 2 3 ... 25225 25226 25227 25228
Data variables:
    Sv                     (frequency, ping_time, range_bin) float64 nan ... ...
    range                  (frequency, ping_time, range_bin) float64 0.0 ... ...
    temperature            (ping_time) float64 6.9 nan nan nan ... nan nan nan
    salinity               (ping_time) float64 33.7 nan nan nan ... nan nan nan
    pressure               (ping_time) float64 240.0 nan nan nan ... nan nan nan
    sound_speed            (ping_time) float64 1.486e+03 nan nan ... nan nan nan
    sound_absorption       (ping_time, frequency) float64 0.02028 ... nan
    sa_correction          object None
    gain_correction        object None
    equivalent_beam_angle  (frequency) float64 -20.7 -20.7